In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

### How do the Claude 3 model family and GPT-4 perform on zero-shot MMLU College Math?

In [8]:
ZeroShotFourOptionResponses = pd.read_csv(
    "../data/ZeroShotFourOptionResponses_20240505205946.csv",
    index_col=0,
)
ZeroShotFourOptionResponses

,question_id,response,model_id,is_correct
id,,,,
89b40ce8-6eeb-4890-8a2c-2339e0aa7d13,0,B,gpt-4-turbo-2024-04-09,True
8ef0d0ab-3e7f-4e93-af0e-c18adfd01981,1,D,gpt-4-turbo-2024-04-09,True
addc10b8-0c0f-407b-a68b-e406a6ab0699,2,A,gpt-4-turbo-2024-04-09,False
35d854e4-c202-4d41-9c4b-78c4356e1271,3,D,gpt-4-turbo-2024-04-09,False
f027e705-89c7-4228-8ac4-a37795c96033,4,C,gpt-4-turbo-2024-04-09,True
...,...,...,...,...
f6a2de3b-0a16-4a71-bcab-34486a7c92bb,95,D,claude-3-opus-20240229,True
d0b4096d-339c-49e4-879e-6b15327e34fa,96,D,claude-3-opus-20240229,False
ede61d4a-09a2-4db4-9038-18c102b42b87,97,C,claude-3-opus-20240229,True


In [9]:
correct_answers_by_model_df = (
    ZeroShotFourOptionResponses.groupby("model_id")["is_correct"].sum().reset_index()
)
correct_answers_by_model_df["model_id"] = pd.Categorical(
    correct_answers_by_model_df["model_id"],
    categories=[
        "gpt-4-turbo-2024-04-09",
        "claude-3-haiku-20240307",
        "claude-3-sonnet-20240229",
        "claude-3-opus-20240229",
    ],
    ordered=True,
)
correct_answers_by_model_df = correct_answers_by_model_df.sort_values("model_id")
correct_answers_by_model_df

,model_id,is_correct
3,gpt-4-turbo-2024-04-09,54
0,claude-3-haiku-20240307,35
2,claude-3-sonnet-20240229,30
1,claude-3-opus-20240229,45


In [10]:
correct_answers_by_model_df["percentage_is_correct"] = (
    correct_answers_by_model_df["is_correct"] / 100
)

fig = px.bar(
    correct_answers_by_model_df,
    x="model_id",
    y="percentage_is_correct",
    orientation="v",
    color="model_id",
    labels={
        "model_id": "Model",
        "percentage_is_correct": "Percentage of correct responses",
    },
    title="Percentage of correct responses by model for MMLU College Math Test",
)

fig.add_shape(
    showlegend=True,
    type="line",
    x0=-0.5,
    x1=3.5,
    y0=0.25,
    y1=0.25,
    line=dict(
        color="black",
        width=1,
        dash="dash",
    ),
    name="random guessing",
)

fig.update_xaxes(showticklabels=False, visible=False)
fig.update_yaxes(tickformat=".0%")
fig.update_layout(
    autosize=True,
    height=500,
)

fig.show()
fig.write_html("../plots/correct_responses_by_model.html")

### Can we select a subset of 10 questions which maximises the gap between the expert model (GPT-4) and the weakest non-expert model (Claude 3 Haiku)?

In [11]:
MMLUMathQuestionsSubset = pd.read_csv(
    "../data/MMLUMathQuestionsSubset_20240505210028.csv",
    index_col=0,
)

In [12]:
questions_subset = ZeroShotFourOptionResponses[
    ZeroShotFourOptionResponses["question_id"].isin(MMLUMathQuestionsSubset.index)
]

correct_answers_on_subset_by_model_df = (
    questions_subset.groupby("model_id")["is_correct"].sum().reset_index()
)
correct_answers_on_subset_by_model_df["model_id"] = pd.Categorical(
    correct_answers_on_subset_by_model_df["model_id"],
    categories=[
        "gpt-4-turbo-2024-04-09",
        "claude-3-haiku-20240307",
        "claude-3-sonnet-20240229",
        "claude-3-opus-20240229",
    ],
    ordered=True,
)
correct_answers_on_subset_by_model_df = (
    correct_answers_on_subset_by_model_df.sort_values("model_id")
)
correct_answers_on_subset_by_model_df

,model_id,is_correct
3,gpt-4-turbo-2024-04-09,10
0,claude-3-haiku-20240307,0
2,claude-3-sonnet-20240229,2
1,claude-3-opus-20240229,6


In [13]:
correct_answers_on_subset_by_model_df["percentage_is_correct"] = (
    correct_answers_on_subset_by_model_df["is_correct"] / 10
)

fig = px.bar(
    correct_answers_on_subset_by_model_df,
    x="model_id",
    y="percentage_is_correct",
    orientation="v",
    color="model_id",
    labels={
        "model_id": "Model",
        "percentage_is_correct": "Percentage of correct responses",
    },
    title="Percentage of correct responses by model for selected subset of MMLU College Math Test",
)
fig.add_shape(
    showlegend=True,
    type="line",
    x0=-0.5,
    x1=3.5,
    y0=0.25,
    y1=0.25,
    line=dict(
        color="black",
        width=1,
        dash="dash",
    ),
    name="random guessing",
)


fig.update_xaxes(showticklabels=False, visible=False)
fig.update_yaxes(tickformat=".0%")
fig.update_layout(
    autosize=True,
    height=500,
)

fig.show()
fig.write_html("../plots/correct_responses_on_subset_by_model.html")

### What are the distributions of argument lengths for each judge model?

In [14]:
SelectedOptionArgumentResponses = pd.read_csv(
    "../data/SelectedOptionArgumentResponses_20240505212905.csv",
    index_col=0,
)

In [15]:
argument_length_traces = []
argument_length_medians = {}
for requested_response_length in ["short", "medium", "long"]:
    data = SelectedOptionArgumentResponses[
        SelectedOptionArgumentResponses["requested_response_length"]
        == requested_response_length
    ]["argument"].str.len()
    argument_length_traces.append(
        go.Histogram(
            x=data,
            name=requested_response_length,
            opacity=0.75,
            nbinsx=10,
        )
    )
    # Add a vertical line for the median
    argument_length_median = np.median(data)
    argument_length_medians[requested_response_length] = argument_length_median
    argument_length_traces.append(
        go.Scatter(
            x=[argument_length_median, argument_length_median],
            y=[0, 25],
            mode="lines",
            name=f"{requested_response_length} median",
            showlegend=True,
            line=dict(color="black", width=1, dash="dash"),
        )
    )

layout = go.Layout(
    title="Distributions of the lengths of arguments generated by GPT-4",
    xaxis=dict(title="Argument length in characters"),
    barmode="overlay",
    autosize=True,
    height=500,
)
fig = go.Figure(data=argument_length_traces, layout=layout)
fig.show()
fig.write_html("../plots/argument_length_distributions.html")

### How does argument classification accuracy vary across the Claude 3 model family?

In [16]:
BaselineArgumentClassificationResponses = pd.read_csv(
    "../data/BaselineArgumentClassificationResponses_20240528230036.csv", index_col=0
)
BaselineArgumentClassificationResponses

,question_id,selected_option_index,model_id,classification,classification_summary
id,,,,,
6f9cb595-5177-40dc-8152-b5c871ab6a10,1,0,claude-3-haiku-20240307,"The given answer ""0"" is incorrect.\n\nUp to is...",i
3023e506-c6ef-4bc4-af3e-956295c53620,1,0,claude-3-sonnet-20240229,"The given answer ""0"" is incorrect.\n\nExplanat...",i
17129771-9db4-4f15-a702-67b4b9265f30,1,0,claude-3-opus-20240229,To determine if the given answer is correct or...,i
1f17735d-ed0c-49e0-8431-425bb1604137,1,1,claude-3-haiku-20240307,The given answer is:\n\nAnswer: 1\n\nTo determ...,i
c1e640c7-a800-4358-bc36-9091bb498aa4,1,1,claude-3-sonnet-20240229,Incorrect.\n\nThe given answer is incorrect fo...,i
...,...,...,...,...,...
87fdd799-e165-4249-9e77-220a48b478e1,42,2,claude-3-sonnet-20240229,To determine if the given answer is correct or...,i
dcf430f3-f89d-43f1-bf93-248d968ed9a6,42,2,claude-3-opus-20240229,To determine if the given answer is correct or...,i
ea0393c8-efa2-49a3-a99c-a26e58901560,42,3,claude-3-haiku-20240307,To determine if the given answer is correct or...,i


In [17]:
baseline_classifications_with_questions = pd.merge(
    BaselineArgumentClassificationResponses,
    MMLUMathQuestionsSubset.reset_index()[["id", "correct_option_index"]],
    left_on="question_id",
    right_on="id",
)
baseline_classifications_with_questions["option_is_correct"] = (
    baseline_classifications_with_questions["selected_option_index"]
    == baseline_classifications_with_questions["correct_option_index"]
)
baseline_classifications_with_questions["is_correct"] = (
    baseline_classifications_with_questions["option_is_correct"]
    == (baseline_classifications_with_questions["classification_summary"] == "c")
)
baseline_classifications_with_questions[
    ["question_id", "selected_option_index", "model_id", "is_correct"]
]
baseline_classifications_with_questions["requested_response_length"] = "no argument"

In [18]:
ArgumentClassificationResponses = pd.read_csv(
    "../data/ArgumentClassificationResponses_20240529231504.csv", index_col=0
)

In [19]:
classifications_with_argument_length = pd.merge(
    ArgumentClassificationResponses,
    SelectedOptionArgumentResponses.reset_index()[
        ["id", "requested_response_length", "selected_option_index", "question_id"]
    ],
    left_on="argument_id",
    right_on="id",
)
classification_with_argument_length_and_questions = pd.merge(
    classifications_with_argument_length,
    MMLUMathQuestionsSubset.reset_index()[["id", "correct_option_index"]],
    left_on="question_id",
    right_on="id",
)
classification_with_argument_length_and_questions["option_is_correct"] = (
    classification_with_argument_length_and_questions["selected_option_index"]
    == classification_with_argument_length_and_questions["correct_option_index"]
)
classification_with_argument_length_and_questions["is_correct"] = (
    classification_with_argument_length_and_questions["option_is_correct"]
    == (
        classification_with_argument_length_and_questions["classification_summary"]
        == "c"
    )
)
classification_with_argument_length_and_questions[
    [
        "question_id",
        "selected_option_index",
        "model_id",
        "requested_response_length",
        "is_correct",
    ]
]

,question_id,selected_option_index,model_id,requested_response_length,is_correct
0,1,0,claude-3-sonnet-20240229,short,False
1,1,1,claude-3-sonnet-20240229,short,False
2,1,2,claude-3-sonnet-20240229,short,False
3,1,3,claude-3-sonnet-20240229,short,True
4,8,0,claude-3-sonnet-20240229,short,False
...,...,...,...,...,...
355,29,3,claude-3-haiku-20240307,medium,True
356,35,0,claude-3-haiku-20240307,medium,True
357,42,0,claude-3-haiku-20240307,medium,True
358,22,2,claude-3-haiku-20240307,long,True


In [20]:
classification_with_argument_length_and_questions_with_baseline = pd.concat(
    [
        baseline_classifications_with_questions,
        classification_with_argument_length_and_questions,
    ]
)[
    [
        "question_id",
        "selected_option_index",
        "model_id",
        "requested_response_length",
        "is_correct",
    ]
]
classification_with_argument_length_and_questions_with_baseline

,question_id,selected_option_index,model_id,requested_response_length,is_correct
0,1,0,claude-3-haiku-20240307,no argument,True
1,1,0,claude-3-sonnet-20240229,no argument,True
2,1,0,claude-3-opus-20240229,no argument,True
3,1,1,claude-3-haiku-20240307,no argument,True
4,1,1,claude-3-sonnet-20240229,no argument,True
...,...,...,...,...,...
355,29,3,claude-3-haiku-20240307,medium,True
356,35,0,claude-3-haiku-20240307,medium,True
357,42,0,claude-3-haiku-20240307,medium,True
358,22,2,claude-3-haiku-20240307,long,True


In [21]:
argument_length_medians["no argument"] = 0

In [22]:
classification_with_argument_length_and_questions_with_baseline.groupby(
    ["model_id", "requested_response_length"]
)["is_correct"].sum()

model_id                  requested_response_length
claude-3-haiku-20240307   long                         12
                          medium                       15
                          no argument                  26
                          short                        14
claude-3-opus-20240229    long                         21
                          medium                       25
                          no argument                  35
                          short                        24
claude-3-sonnet-20240229  long                         14
                          medium                       17
                          no argument                  25
                          short                        19
Name: is_correct, dtype: int64

In [23]:
requested_response_lengths = ["no argument", "short", "medium", "long"]
model_ids = [
    "claude-3-haiku-20240307",
    "claude-3-sonnet-20240229",
    "claude-3-opus-20240229",
]

In [24]:
scatter_plots = []
data_by_requested_response_length = {
    requested_response_length: classification_with_argument_length_and_questions_with_baseline[
        classification_with_argument_length_and_questions_with_baseline[
            "requested_response_length"
        ]
        == requested_response_length
    ]
    for requested_response_length in requested_response_lengths
}

for requested_response_length, data in data_by_requested_response_length.items():
    scatter_plots.append(
        go.Scatter(
            name=requested_response_length,
            x=model_ids,
            y=[
                data[data["model_id"] == model_id]["is_correct"].sum() / 40
                for model_id in model_ids
            ],
            mode="markers+lines",
            marker=dict(symbol="cross"),
        )
    )

scatter_plots.append(
    go.Scatter(
        x=model_ids,
        y=[0.5, 0.5, 0.5],
        mode="lines",
        name="random guessing",
        line=dict(color="black", dash="dash"),
    )
)
scatter_plots.append(
    go.Scatter(
        x=model_ids,
        y=[0.25, 0.25, 0.25],
        mode="lines",
        name="always guessing correct",
        line=dict(color="black", dash="dashdot"),
    )
)

fig = go.Figure(data=scatter_plots)
fig.update_yaxes(tickformat=".0%")
fig.update_layout(
    title="Judge accuracy across the Claude 3 model family for different argument lengths",
    yaxis_title="Percentage of accurate judgments",
    xaxis_title="Model",
    autosize=True,
    height=500,
)

fig.show()
fig.write_html("../plots/judge_accuracy_by_model.html")

### How does argument classification accuracy vary with argument length?

In [25]:
scatter_plots = []
data_by_model_id = {
    model_id: classification_with_argument_length_and_questions_with_baseline[
        classification_with_argument_length_and_questions_with_baseline["model_id"]
        == model_id
    ]
    for model_id in model_ids
}

for model_id, data in data_by_model_id.items():
    scatter_plots.append(
        go.Scatter(
            name=model_id,
            x=sorted(argument_length_medians.values()),
            y=[
                data[data["requested_response_length"] == requested_response_length][
                    "is_correct"
                ].sum()
                / 40
                for requested_response_length in requested_response_lengths
            ],
            mode="markers+lines",
            marker=dict(symbol="cross"),
        )
    )

scatter_plots.append(
    go.Scatter(
        x=sorted(argument_length_medians.values()),
        y=[0.5, 0.5, 0.5, 0.5],
        mode="lines",
        name="random guessing",
        line=dict(color="black", dash="dash"),
    )
)
scatter_plots.append(
    go.Scatter(
        x=sorted(argument_length_medians.values()),
        y=[0.25, 0.25, 0.25, 0.25],
        mode="lines",
        name="always guessing correct",
        line=dict(color="black", dash="dashdot"),
    )
)

fig = go.Figure(data=scatter_plots)
fig.update_yaxes(tickformat=".0%")
fig.update_layout(
    title="Judge accuracy for different argument lengths across the Claude 3 model family",
    yaxis_title="Percentage of accurate judgments",
    xaxis_title="Median argument length in characters",
    autosize=True,
    height=500,
)

fig.show()
fig.write_html("../plots/judge_accuracy_by_argument_length.html")

### Does debate improve zero-shot performance?

In [26]:
ordered_claude_model_ids = [
    "claude-3-haiku-20240307",
    "claude-3-sonnet-20240229",
    "claude-3-opus-20240229",
]

In [27]:
ZeroShotTwoOptionResponses = pd.read_csv(
    "../data/ZeroShotTwoOptionResponses_20240529232343.csv", index_col=0
)
ZeroShotTwoOptionResponses["correct_option"] = ZeroShotTwoOptionResponses[
    "ordering"
].apply(lambda ord: "A" if ord == "correct_first" else "B")
ZeroShotTwoOptionResponses["is_correct"] = (
    ZeroShotTwoOptionResponses["response_summary"]
    == ZeroShotTwoOptionResponses["correct_option"]
)

In [28]:
two_option_zero_shot_by_model_df = (
    ZeroShotTwoOptionResponses.groupby("model_id")["is_correct"].sum().reset_index()
)
two_option_zero_shot_by_model_df["model_id"] = pd.Categorical(
    two_option_zero_shot_by_model_df["model_id"],
    categories=ordered_claude_model_ids,
    ordered=True,
)
two_option_zero_shot_by_model_df = two_option_zero_shot_by_model_df.sort_values(
    "model_id"
)
two_option_zero_shot_by_model_df["percentage_is_correct"] = (
    two_option_zero_shot_by_model_df["is_correct"] / 20
)
two_option_zero_shot_by_model_df

,model_id,is_correct,percentage_is_correct
0,claude-3-haiku-20240307,14,0.7
2,claude-3-sonnet-20240229,12,0.6
1,claude-3-opus-20240229,18,0.9


In [29]:
TwoOptionDebateResponses = pd.read_csv(
    "../data/TwoOptionDebateResponses_20240529235651.csv", index_col=0
)
TwoOptionDebateResponses["correct_option"] = TwoOptionDebateResponses["ordering"].apply(
    lambda ord: "A" if ord == "correct_first" else "B"
)
TwoOptionDebateResponses["is_correct"] = (
    TwoOptionDebateResponses["response_summary"]
    == TwoOptionDebateResponses["correct_option"]
)

In [30]:
two_option_debate_by_model_df = (
    TwoOptionDebateResponses.groupby("model_id")["is_correct"].sum().reset_index()
)
two_option_debate_by_model_df["model_id"] = pd.Categorical(
    two_option_debate_by_model_df["model_id"],
    categories=ordered_claude_model_ids,
    ordered=True,
)
two_option_debate_by_model_df = two_option_debate_by_model_df.sort_values("model_id")
two_option_debate_by_model_df["percentage_is_correct"] = (
    two_option_debate_by_model_df["is_correct"] / 20
)
two_option_debate_by_model_df

,model_id,is_correct,percentage_is_correct
0,claude-3-haiku-20240307,13,0.65
2,claude-3-sonnet-20240229,12,0.60
1,claude-3-opus-20240229,16,0.80


In [31]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=ordered_claude_model_ids,
        y=two_option_zero_shot_by_model_df["percentage_is_correct"],
        name="no arguments",
    )
)
fig.add_trace(
    go.Bar(
        x=ordered_claude_model_ids,
        y=two_option_debate_by_model_df["percentage_is_correct"],
        name="single argument for each side",
    )
)

fig.add_shape(
    showlegend=True,
    type="line",
    x0=-0.5,
    x1=2.5,
    y0=0.5,
    y1=0.5,
    line=dict(
        color="black",
        width=1,
        dash="dash",
    ),
    name="random guessing",
)

fig.update_yaxes(tickformat=".0%")
fig.update_layout(
    barmode="group",
    title="Judge accuracy across the Claude 3 model family with/without debate",
    yaxis_title="Percentage of accurate judgments",
    xaxis_title="Model",
    autosize=True,
    height=500,
)

fig.show()
fig.write_html("../plots/judge_accuracy_by_model_with_debate.html")